<a href="https://colab.research.google.com/github/IzhbuldinaDaria2002/-/blob/main/%D0%BE%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Загрузка
df_orders = pd.read_csv('web_app_orders_traffic_all.csv')
df_app_sessions = pd.read_csv('app_traffic_all.csv')

# Web-сессии
web_sessions = df_orders[df_orders['platform'] == 'web'][
    ['date', 'country', 'channel_1', 'channel_2', 'sessions']
].copy()
web_sessions.rename(columns={'sessions': 'web_sessions'}, inplace=True)

# Заказы и доход (суммарно)
orders_revenue = df_orders.groupby(
    ['date', 'country', 'channel_1', 'channel_2'],
    as_index=False
).agg({
    'orders': 'sum',
    'revenue': 'sum'
})

# App-сессии
app_sessions = df_app_sessions[['date', 'country', 'channel_1', 'channel_2', 'sessions']].copy()
app_sessions.rename(columns={'sessions': 'app_sessions'}, inplace=True)

# Объединение
merged = pd.merge(orders_revenue, web_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')
merged = pd.merge(merged, app_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')

# Заполнение NaN
merged.fillna({'web_sessions': 0, 'app_sessions': 0, 'orders': 0, 'revenue': 0}, inplace=True)

# Общие сессии
merged['sessions'] = merged['web_sessions'] + merged['app_sessions']

# Сохранение
merged.to_csv('merged_data.csv', index=False)